In [42]:
# 시스템
import os
import sys
import re
# 크롤링
import urllib.request
import requests
import json
from scrapy.http import TextResponse
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [8]:
# 태그 데이터를 삭제하는 코드
def tag_remove(word):

    word = re.sub('(<([^>]+)>)', '', word)

    return word

# 한글만 남기는 코드 -> 태그만 삭제해야함
def preprocess_sentence_kr(w):
    w = w.strip()
    w = re.sub(r"[^0-9가-힣?.!,¿]+", " ", w) # \n도 공백으로 대체해줌
    w = w.strip()
    return w

# 한글 영어만 남기는 코드 -> 태그만 삭제해야함
def preprocess_sentence(w):
    w = w.strip()
    w = re.sub(r"[^0-9가-힣a-zA-Z?.!,¿]+", " ", w) # \n도 공백으로 대체해줌
    w = w.strip()
    return w

In [268]:
def blog_content_parsing(url):
    blog_req = requests.get(url)
    blog_bs = BeautifulSoup(blog_req.text, "html.parser")
    
    result = []
    quote = []
    
    quote_cnt = 0
    img_cnt = 0
    blog_text, content, first_img, last_img = "", "" ,"", ""
    try:
        # 현재 여기서 안되는것은 스마트에디터 3로 작성된 경우이다.
        # 이미지 내용 크롤링
        img_craw = blog_bs.select(".se-module-image")
        img_cnt = len(img_craw)

        if img_cnt != 0: # 사진이 있는 경우만 크롤링
            try:
                first_img = img_craw[0].select_one(".se-module-image-link > img")["data-lazy-src"]
                last_img = img_craw[img_cnt-1].select_one(".se-module-image-link > img")["data-lazy-src"]

            except:
                first_img = img_craw[0].select_one(".se-module-image-link > img")["src"]
                last_img = img_craw[img_cnt-1].select_one(".se-module-image-link > img")["src"]
        # 글 내용 크롤링
        blog_text = blog_bs.select(".se-text")
        for i in blog_text:
            for j in i.select(".se-module-text > p"):
                txt = j.text
                if txt != "\u200b":
                    result.append(txt)
        content = " ".join(result)

        for i in blog_bs.select(".se-quote"):
            quote.append(i.text)
            quote_cnt += 1
    except:
        return False, content, len(content), len(blog_text) ,quote, quote_cnt,  first_img, last_img,  img_cnt
        
    return True, content,len(content), len(blog_text) ,quote, quote_cnt, first_img, last_img,  img_cnt

In [269]:
# blog가 네이버 블로그인지 아닌지 판별하는 코드

def naver_blog_check(code):
    
    if code[0].isdigit():
        return False
    else:
        return True


# 모든 단어를 하나씩 나누어서 확인하자.
# 

def naver_title_check(search_word, title):
     
    # search_word = preprocess_sentence_kr(search_word)
    
    for words in search_word.split():
        for word in words:
            if word.lower() not in title.lower():
                return True

In [270]:
# 이름이 들어오면 해당 내용을 이용하여 블로그를 파싱한다.

# search_word : 어떤 제품을 검색하는지
# repeat_num : 해당 제품이 제목에서 몇번 안나올때까지 검색을 진행할 지
def item_parsing(search_word, start, display, repeat_num):
   
    blog_search = naver_api(search_word, start, display)
    print("start ++++++++++++++++++++")
    # result : url
    # writer : 작성자 이름
    # code : 블로그 코드
    # title : 블로그 제목
    # description : 블로그 요약 설명
    blog_result = []
    blog_writer = []
    blog_code = []
    blog_title = []
    blog_description = []
    repeat = True
    cnt = 0
    # 제목에서 해당 제품의 이름이 5번 연속 안나온다면 패스
    while repeat:
        
        for blog in blog_search["items"]: 
             # 블로그 코드
            code = blog["link"][blog["link"].find("logNo=") + 6:]
            if naver_blog_check(code): # 네이버 블로그가 아닌경우 다음으로 넘기기
                continue

            # 이름
            writer = blog["bloggerlink"][23:]

            # 제목
            title = tag_remove(blog["title"])
            
            if naver_title_check(search_word, title):
                continue

            # 블로그 요약
            description = tag_remove(blog["description"])
            ## 데이터 추가하는 곳
            # 블로그 코드 추가
            blog_code.append(code)
            # 이름 추가
            blog_writer.append(writer)
            # 제목 추가
            blog_title.append(title)
            # 블로그 요약 추가
            blog_description.append(description)
            # 본문 크롤링을 위한 URL주소 추가
            blog_result.append(f"https://blog.naver.com/PostView.naver?blogId={writer}&logNo={code}")
        if start + display <= blog_search["total"]:
            start += display
            blog_search = naver_api(search_word, start, display)
            print(f"start = {start}")
            print("repeat start ++++++++++++++++++++")
        else:
            print("repeat End")
            repeat = False
    return blog_result, blog_writer, blog_code, blog_title, blog_description

In [12]:
def naver_api(search_word, start, display):
    client_id = "GvNa2sBgFDA6v7ujnaz0"
    client_secret = "Yo0jOskXlZ"
    
    encText = urllib.parse.quote(search_word)
    
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText # json 결과
    url += f"&start={start}"
    url += f"&display={display}"

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
    else:
        print("Error Code:" + rescode)
    return json.loads(response_body.decode('utf-8'))

In [13]:
def search_word(word):
    result = []
    append_data = ["후기", "리뷰"]
    for data in append_data:
        result.append(word + " " + data)
        
    return result

    

In [271]:
df = pd.DataFrame()
words = search_word("WF-1000XM4")
url, title = [], []
for word in words:
    blog_result, blog_writer, blog_code, blog_title, blog_description = item_parsing(word, 1, 100, 3)
    url += blog_result
    title += blog_title
    # description += blog_description
    
df["url"] = url
df["title"] = title
# df["description"] = description

df = df.drop_duplicates(["url"]).reset_index(drop=True)


start ++++++++++++++++++++
start = 101
repeat start ++++++++++++++++++++
start = 201
repeat start ++++++++++++++++++++
start = 301
repeat start ++++++++++++++++++++
repeat End
start ++++++++++++++++++++
start = 101
repeat start ++++++++++++++++++++
start = 201
repeat start ++++++++++++++++++++
start = 301
repeat start ++++++++++++++++++++
start = 401
repeat start ++++++++++++++++++++
repeat End


In [272]:
content_list,content_cnt_list,content_line_list, quote_list, quote_cnt_list, first_img_list, last_img_list, img_cnt_list = [], [], [], [], [], [], [], []
for i in tqdm(range(len(df))):
    _, content, content_cnt, content_line, quote, quote_cnt, first_img, last_img, img_cnt =  blog_content_parsing(df.loc[i, "url"])

    content_list.append(content)
    content_cnt_list.append(content_cnt)
    content_line_list.append(content_line)
    quote_list.append(quote)
    quote_cnt_list.append(quote_cnt)
    first_img_list.append(first_img)
    last_img_list.append(last_img)
    img_cnt_list.append(img_cnt)

df["content"] = content_list
df["content_cnt"] = content_cnt_list
df["content_line"] = content_line_list
df["quote"] = quote_list
df["quote_cnt"] = quote_cnt_list
df["first_img"] = first_img_list
df["last_img"] = last_img_list
df["img_cnt"] = img_cnt_list


100%|██████████| 121/121 [00:38<00:00,  3.18it/s]


In [274]:
def search_word(word):
    result = []
    append_data = ["후기", "리뷰"]
    for data in append_data:
        result.append(word + " " + data)
        
    return result

    

In [278]:
def df_keyword_contains(df):

    keyword_contains = ["허락", "내돈내산", "리얼후기" ,"협찬", "체험단","coupa.ng", "<", ">", "♡", "♥", "구매후기"]
    keyword_cnt = ["솔직", "비교", "ㅋ", "ㅋㅋ" ,"ㅋㅋㅋ", "ㅋㅋㅋㅋ", "...","....", "ㅜ", "ㅜㅜ", "ㅜㅜㅜ", "ㅜㅜㅜㅜ", "ㅠ", "ㅠㅠ", "ㅠㅠㅠ", "ㅠㅠㅠㅠ"]
    keyword_badword = ["개좋다", "개좋음", "개멋짐", "개빠름", "개큼", "존나", "걍", "씹창"]
    
    for key in keyword_cnt:
        df[key + " 빈도 수"] = 0
    df["비속어 빈도 수"] = 0

    for i in range(len(df)):
        content = df.loc[i, "content"]
        for key in keyword_contains:
            if key in content:
                df.loc[i, key+" 키워드"] = "1"
                
            else:
                df.loc[i, key+" 키워드"] = "0"
        
        for cont in content.split():
            for key in keyword_cnt:
                if key in cont:
                    df.loc[i, key + " 빈도 수"] += 1
            
            for bad in keyword_badword:
                if bad in cont:
                    df.loc[i, "비속어 빈도 수"] += 1
    return df



In [279]:
df_keyword_contains(df)

,url,title,content,content_cnt,content_line,quote,quote_cnt,first_img,last_img,img_cnt,...,내돈내산 키워드,리얼후기 키워드,협찬 키워드,체험단 키워드,coupa.ng 키워드,< 키워드,> 키워드,♡ 키워드,♥ 키워드,구매후기 키워드
0,https://blog.naver.com/PostView.naver?blogId=r...,소니 WF-1000XM4 노이즈 캔슬링 이어폰 개봉 후기...,지난 몇 주간 소니 WF-1000XM4에 대한 사용 후기를 남겼는데 개봉기를 빼놓을...,3193,30,[],0,https://postfiles.pstatic.net/MjAyMTExMTdfMjcw...,https://postfiles.pstatic.net/MjAyMTExMThfMjY0...,40,...,0,0,0,0,0,0,0,0,0,0
1,https://blog.naver.com/PostView.naver?blogId=f...,무선 노이즈 캔슬링 이어폰 소니 WF-1000XM4 간단 후기,노이즈 캔슬링 이어폰의 왕의 귀환! 소니 WF-1000XM4가 발매가 되었습니다....,3452,15,[],0,https://postfiles.pstatic.net/MjAyMTA3MDdfMTYy...,https://postfiles.pstatic.net/MjAyMTA3MDdfMTAw...,18,...,0,0,0,0,0,0,0,0,0,0
2,https://blog.naver.com/PostView.naver?blogId=s...,준하는 노이즈캔슬링 소니 WF-1000XM4 무선이어폰 후기,드디어 나왔습니다! 크!! 기다리고 기다렸던 소니 WF-1000XM4 노이즈캔슬링 ...,3315,19,[],0,https://postfiles.pstatic.net/MjAyMTA4MjZfNDMg...,https://postfiles.pstatic.net/MjAyMDAzMjVfMjM4...,20,...,0,0,0,0,0,0,0,0,0,0
3,https://blog.naver.com/PostView.naver?blogId=l...,소니 무선 노이즈캔슬링 이어폰 WF-1000XM4 내돈내산 리뷰 후기,소니에서 이번에 새로 정식 출시한 WF-1000XM4는 전작 WF-1000XM3에서...,2152,14,[],0,https://postfiles.pstatic.net/MjAyMTA2MjlfMjA5...,https://postfiles.pstatic.net/MjAyMTA2MjlfNTQg...,12,...,0,0,0,0,0,0,0,0,0,0
4,https://blog.naver.com/PostView.naver?blogId=x...,"[내돈내산 후기] 음질, 노캔 최강 소니 WF-1000XM4","안녕하세요, 밝은불꽃입니다! 요즘 날씨가 더워서 헤드폰 대신 블루투스 이어폰을 들...",3203,25,"[간단한 음질 비교?, 아무튼 결론]",2,https://postfiles.pstatic.net/MjAyMTA2MzBfMjcx...,https://postfiles.pstatic.net/MjAyMTA2MzBfMjYy...,27,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,https://blog.naver.com/PostView.naver?blogId=k...,"소니 WF-1000XM4 한달만에 도착, 간단리뷰",기다리고 기다리던 XM4가 출시되었다는 소식을 듣고 부랴부랴 예약을 했던게 벌써 한...,3402,23,[],0,https://postfiles.pstatic.net/MjAyMTA4MTFfMjIg...,https://postfiles.pstatic.net/MjAyMTA4MTFfNTAg...,35,...,0,0,0,0,0,0,0,0,0,0
117,https://blog.naver.com/PostView.naver?blogId=l...,소니 WF-1000XM4 리뷰 노이즈 캔슬링 추천 신제품 정보,,0,0,[],0,,,0,...,0,0,0,0,0,0,0,0,0,0
118,https://blog.naver.com/PostView.naver?blogId=l...,"소니 WF-1000XM4, 하이엔드 무선 인이어 헤드폰 리뷰",,0,0,[],0,,,0,...,0,0,0,0,0,0,0,0,0,0
119,https://blog.naver.com/PostView.naver?blogId=b...,블루투스 이어폰 끝판왕 SONY WF-1000XM4 + 모든 기능 리뷰,안드로이드폰 최강조합 블루투스 이어폰 최신 V1 프로세서 탑재되어있으며 자그마치 6...,1880,27,[],0,https://postfiles.pstatic.net/MjAyMTEyMDdfMjM4...,https://postfiles.pstatic.net/MjAyMTEyMDhfNTMg...,30,...,0,0,0,0,0,0,0,0,0,0


In [283]:
df.columns

Index(['url', 'title', 'content', 'content_cnt', 'content_line', 'quote',
       'quote_cnt', 'first_img', 'last_img', 'img_cnt', '솔직 빈도 수', '비교 빈도 수',
       'ㅋ 빈도 수', 'ㅋㅋ 빈도 수', 'ㅋㅋㅋ 빈도 수', 'ㅋㅋㅋㅋ 빈도 수', '... 빈도 수', '.... 빈도 수',
       'ㅜ 빈도 수', 'ㅜㅜ 빈도 수', 'ㅜㅜㅜ 빈도 수', 'ㅜㅜㅜㅜ 빈도 수', 'ㅠ 빈도 수', 'ㅠㅠ 빈도 수',
       'ㅠㅠㅠ 빈도 수', 'ㅠㅠㅠㅠ 빈도 수', '비속어 빈도 수', '허락 키워드', '내돈내산 키워드', '리얼후기 키워드',
       '협찬 키워드', '체험단 키워드', 'coupa.ng 키워드', '< 키워드', '> 키워드', '♡ 키워드', '♥ 키워드',
       '구매후기 키워드', '광고 분류1', '광고 분류2'],
      dtype='object')

In [284]:
str(0) == str("0")

True

In [287]:
def df_check_ad(df):
    df["광고 분류1"] = 9
    df["광고 분류2"] = 9
    
    
    pure_review = ["허락 키워드", "내돈내산 키워드", "리얼후기 키워드", "솔직 빈도 수", "비교 빈도 수"]
    reward_review = ["coupa.ng 키워드"]
    advertise_review = ["협찬 키워드", "체험단 키워드"]
    items = [pure_review, reward_review, advertise_review]
    
    for i in range(len(df)):

        for idx,  item in enumerate(items):

            if idx == 0:
                for it in item:
                    if str(df.loc[i, it]) != "0":
                        df.loc[i, "광고 분류1"] = 0
                        df.loc[i, "광고 분류2"] = 0
                        break

            elif idx == 1:
                for it in item:
                    if df.loc[i, it] != "0":
                        df.loc[i, "광고 분류1"] = 1
                        df.loc[i, "광고 분류2"] = 1
                        break

            elif idx == 2:
                for it in item:
                    if df.loc[i, it] != "0":
                        df.loc[i, "광고 분류1"] = 1
                        df.loc[i, "광고 분류2"] = 2
                        break

        

In [288]:
df_check_ad(df)

In [226]:
df2 = df.copy()

In [293]:
df.to_csv("소1234.csv", index= False,encoding="utf-8-sig")